In [0]:
%md

# schema (verified)
https://docs.google.com/spreadsheets/d/1ueWwXolJK_mYwXVLajhIyOWjxSUwWVjnbEfX3011RSw/edit#gid=885079034
https://docs.google.com/spreadsheets/d/1ueWwXolJK_mYwXVLajhIyOWjxSUwWVjnbEfX3011RSw/edit#gid=1590688866


# raw
aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobileweb.domain-retention.v4/
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/


# unified
## wide
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.retention.v6/
## narrow
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.retention.original.v6/


# snowflake
### narrow
snowflake:    AA_INTELLIGENCE_PRODUCTION.ADL_USAGE_PAID.FACT_USAGE_RETENTION_DAYS_V1_CLUSTER_BY_DATE
snowflake:宽表   AA_INTELLIGENCE_PRODUCTION.ADL_USAGE_PAID.FACT_USAGE_RETENTION_V1_CLUSTER_BY_DATE


### question
1. why retention need to add MAU??




In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 

-- adhoc

select country_code,device_code,  retention_days, estimate from plproxy.execute_select_nestloop(\$proxy\$ 
    select country_code,device_code, retention_days, estimate
    from mw.domain_retention_m
    where 
        date  = '2020-08-31' and domain_id=70010000000003 and country_code='us' and device_code='ip'
\$proxy\$) tpl (country_code char, device_code char, retention_days integer, estimate real ) ;

-- domain_id, granularity, date, country_code, device_code, retention_days, estimate


-- total
select sum(count_a) as app_count, sum(sum_d) as sum_d, sum(sum_e) as sum_e from plproxy.execute_select_nestloop(\$proxy\$ 
    select count(*) as count_a, sum(kpi) as sum_d, sum(estimate) as sum_e
    from rt.app_monthly
    where 
        date  = '2020-08-31'
\$proxy\$) tpl (count_a bigint, sum_d bigint, sum_e double precision ) ;


select sum(count_a) as mw_count, sum(sum_d) as sum_d, sum(sum_e) as sum_e from plproxy.execute_select_nestloop(\$proxy\$ 
    select count(*) as count_a, sum(retention_days) as sum_d, sum(estimate) as sum_e
    from mw.domain_retention_m
    where 
        date  = '2020-08-31'
\$proxy\$) tpl (count_a bigint, sum_d bigint, sum_e real ) ;




EOF


#estimate DOUBLE PRECISION


In [0]:

# s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobileweb.domain-retention.v4/fact/granularity=m/month=202008/date=2020-08-31/
# s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/granularity=monthly/month=2020-08/

# spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobileweb.domain-retention.v4/fact/granularity=m/month=202008/date=2020-08-31/").createOrReplaceTempView("dump_retention_mw_original")
spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/granularity=monthly/month=2020-08/").createOrReplaceTempView("dump_retention_app_original")

spark.sql("select kpi, count(*) as count_a, sum(kpi) as sum_d, sum(estimate) as sum_e from dump_retention_app_original where date = '2020-08-31' group by kpi order by kpi asc ").show()
# spark.sql("select * from dump_retention_original where date  = '2020-08-31' and domain_id=70010000000003 and country_code='us' and device_code='ip' order by retention_days asc").show()
# df2 = spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/granularity=monthly/month=2020-08/")
# print(df2.count())


In [0]:

# s3://b2c-prod-data-pipeline-unified-usage/unified/usage.retention.v6/fact/granularity=monthly/date=2020-08-31/
# s3://b2c-prod-data-pipeline-unified-usage/unified/usage.retention.original.v6/fact/granularity=monthly/date=2020-08-31/

#################################################################################################
df2 = spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-usage/unified/usage.retention.original.v6/fact/granularity=monthly/date=2020-08-31/").createOrReplaceTempView("retention_original")
spark.sql("select count(*) as app_count, sum(retention_days) as sum_d, sum(estimate) as sum_e from retention_original where product_type_key=0 ").show(100, False)
spark.sql("select count(*) as app_count, sum(retention_days) as sum_d, sum(estimate) as sum_e from retention_original where product_type_key=1 ").show(100, False)



#################################################################################################
df1 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-usage/unified/usage.retention.v6/fact/granularity=monthly/date=2020-08-31/").createOrReplaceTempView("retention")
spark.sql("select count(*) from retention").show(100, False)

df1.show(20)
df2.show(20)


In [0]:


spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_name=app/granularity=monthly/date=2020-08-31/").createOrReplaceTempView("usage_app_monthly")
spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-usage/unified/usage.retention.v6/fact/granularity=monthly/date=2020-08-31/").createOrReplaceTempView("retention_monthly")

spark.sql("select * from retention_monthly").show(10, False)
spark.sql("select * from retention_monthly where product_key=20600000009072 and country_code='US' and date_key='20200831'").show(100, False)



In [0]:
%%sh

# aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobileweb.domain-retention.v4/fact/granularity=m/month=202008/date=2020-08-31/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/granularity=monthly/month=2020-08/device_id=1001/store_id=10/

# aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.retention.v6/fact/granularity=monthly/date=2020-08-31/
## narrow
# aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v2/fact/granularity=monthly/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v2/fact/granularity=weekly/

aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_name=app/granularity=monthly/date=2020-08-31/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_name=domain/granularity=monthly/date=2020-08-31/


# aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.basic-kpi.v6/fact/granularity=monthly/



                        #   PRE usage.seg-by-demographics.v6/
                        #   PRE usage.seg-by-product.v6/


